# extract FITS image for LSSTCam   

In [6]:
from lsst.daf.butler import Butler
import pathlib


In [7]:
repo = 'embargo'
instrument = 'LSSTCam'
datasetType = 'post_isr_image'
#datasetType = 'preliminary_visit_image'  # background subtracted calibrated image that detection has run on

collections = ["LSSTCam/runs/nightlyValidation", "LSSTCam/defaults",]

butler = Butler(repo, collections=collections, instrument=instrument)



#uncomment to get the help page
#help(Butler) 

## Find the exposures

BLOCK-T519 is the LSSTCam Local Meridian Observations at 80 deg elevation on day = 20250524

BLOCK-T517 — LSSTCam “Sidereal drive off tests” with and without sidereal tracking on day = 20250522

May 22 and 24 had science quality images of M49, Cosmos and dithered star field


In [32]:

day = 20250522
day2 = 20250524

block = 'BLOCK-T519'  
datasetType = 'post_isr_image'

result = butler.query_datasets( datasetType, limit=None, 
        where = f"(day_obs = {day} OR day_obs = {day2})   AND exposure.science_program='{block}' "

                             )
meridianresults = [r for r in result]

print (f"got {len(meridianresults)} datsets")


got 30316 datsets


In [36]:
#  just wrting out 2 .. 
path = ".."
butler.retrieveArtifacts(destination = path, refs=[meridianresults[2],meridianresults[3]])

[ResourcePath("file:///home/o/omullan/notebooks/LSSTCam/runs/nightlyValidation/19/post_isr_image/20250522/MC_O_20250522_000237/post_isr_image_LSSTCam_r_57_MC_O_20250522_000237_R01_S00_LSSTCam_runs_nightlyValidation_19.fits"),
 ResourcePath("file:///home/o/omullan/notebooks/LSSTCam/runs/nightlyValidation/19/post_isr_image/20250522/MC_O_20250522_000238/post_isr_image_LSSTCam_r_57_MC_O_20250522_000238_R01_S00_LSSTCam_runs_nightlyValidation_19.fits")]

In [40]:

day = 20250522
block = 'BLOCK-T517'  

result = butler.query_datasets( datasetType, limit=None,  where = 
          f"day_obs = {day}   AND exposure.science_program='{block}' "

                             )
nonsiderealresult = [r for r in result]

print (f"got {len(nonsiderealresult)} images")

got 14880 images


In [41]:
#  just wrting out 2 .. 
print (nonsiderealresult[2])
path = ".."
butler.retrieveArtifacts(destination = path, refs=[nonsiderealresult[2],nonsiderealresult[3]])

post_isr_image@{instrument: 'LSSTCam', detector: 0, exposure: 2025052200140, band: 'r', day_obs: 20250522, group: '2025-05-23T00:28:42.492', physical_filter: 'r_57'} [sc=Exposure] (run=LSSTCam/runs/nightlyValidation/19 id=12dba2f8-01db-4679-94d0-2ae43471908e)


[ResourcePath("file:///home/o/omullan/notebooks/LSSTCam/runs/nightlyValidation/19/post_isr_image/20250522/MC_O_20250522_000140/post_isr_image_LSSTCam_r_57_MC_O_20250522_000140_R01_S00_LSSTCam_runs_nightlyValidation_19.fits"),
 ResourcePath("file:///home/o/omullan/notebooks/LSSTCam/runs/nightlyValidation/19/post_isr_image/20250522/MC_O_20250522_000141/post_isr_image_LSSTCam_r_57_MC_O_20250522_000141_R01_S00_LSSTCam_runs_nightlyValidation_19.fits")]

In [18]:
import numpy as np

day = 20250522
target = 'M49'  # change to Prawn etc. to get others
seqnums = np.append( np.arange(158,168), np.arange(138,148))
print (seqnums) 
                        
result = butler.query_datasets( datasetType, limit=None,  where = 
    "day_obs = dayObs AND exposure.target_name = target and exposure.seq_num in (seqnums)",
    bind={"dayObs": day, "target": target ,"seqnums": seqnums.tolist()}
                              )
results = [r for r in result]

print (f"got {len(results)} images")

[158 159 160 161 162 163 164 165 166 167 138 139 140 141 142 143 144 145
 146 147]
got 3720 images


In [19]:
#  just wrting out 2 .. 
path = ".."
butler.retrieveArtifacts(destination = path, refs=[results[2],results[3]], overwrite=True)

[ResourcePath("file:///home/o/omullan/notebooks/LSSTCam/runs/nightlyValidation/19/post_isr_image/20250522/MC_O_20250522_000140/post_isr_image_LSSTCam_r_57_MC_O_20250522_000140_R01_S00_LSSTCam_runs_nightlyValidation_19.fits"),
 ResourcePath("file:///home/o/omullan/notebooks/LSSTCam/runs/nightlyValidation/19/post_isr_image/20250522/MC_O_20250522_000141/post_isr_image_LSSTCam_r_57_MC_O_20250522_000141_R01_S00_LSSTCam_runs_nightlyValidation_19.fits")]

LSSTCam has 202 images per exposure - this will list the exposures. 
The last digits after the daynumber  match the 'Seq. No.' in RubinTV

This is just getting the datasetref for an exposure (this is LSSTcam so that should be 189 images one per detector) unless the corner rafts show up

## If you want to plot one of these 

In [ ]:
import pylab as plt
import lsst.afw.display as afwDisplay

afwDisplay.setDefaultBackend('matplotlib')

r = results[2]

print (r)

img = calexp = butler.get(r)

fig = plt.figure()
display = afwDisplay.Display()
display.scale('linear', 'zscale')
display.mtv(img)
plt.show()

# Files
Now we have Fits files we can use with external code.
We get the list of files we made and process them with SEP (Source Extractor Python) as an example of external code. 
We will make a list of Objects and store that.

In [22]:
import glob
filelist = glob.glob('../LSSTCam/*/*/*/*/*/*/*.fits')
print (filelist)

['../LSSTCam/runs/nightlyValidation/19/post_isr_image/20250522/MC_O_20250522_000141/post_isr_image_LSSTCam_r_57_MC_O_20250522_000141_R01_S00_LSSTCam_runs_nightlyValidation_19.fits', '../LSSTCam/runs/nightlyValidation/19/post_isr_image/20250522/MC_O_20250522_000237/post_isr_image_LSSTCam_r_57_MC_O_20250522_000237_R01_S00_LSSTCam_runs_nightlyValidation_19.fits', '../LSSTCam/runs/nightlyValidation/19/post_isr_image/20250522/MC_O_20250522_000140/post_isr_image_LSSTCam_r_57_MC_O_20250522_000140_R01_S00_LSSTCam_runs_nightlyValidation_19.fits', '../LSSTCam/runs/nightlyValidation/19/post_isr_image/20250522/MC_O_20250522_000238/post_isr_image_LSSTCam_r_57_MC_O_20250522_000238_R01_S00_LSSTCam_runs_nightlyValidation_19.fits']
